# Model Translation + APS Viewer with 2D Circle Markers

This notebook demonstrates how to upload a Revit model with `aps_automation_sdk`, translate it to SVF/SVF2, and add interactive 2D circle markers using the **`Draw2DCircles` plugin** from `aps_viewer_sdk`.

## What you'll learn:
- Upload and translate a Revit model using `aps_automation_sdk` helpers
- Use the **`Draw2DCircles` plugin** (from `aps_viewer_sdk.plugins`) to add interactive circle markers
- Add clickable circles to 2D views that users can place by clicking on the drawing
- Customize circle appearance (radius, color, opacity)

## About the Draw2DCircles Plugin

The `Draw2DCircles` plugin is a **2D-only extension** from `aps_viewer_sdk.plugins` that enables interactive annotation on 2D views. Users can click anywhere in a 2D view (floor plans, elevations, sections) to place circle markers, which is useful for:
- Marking points of interest
- Annotating specific locations
- Creating visual indicators for measurements or notes
- Interactive markup sessions

**Prereqs**
- Env vars: `CLIENT_ID`, `CLIENT_SECRET`
- Dependencies: `aps_viewer_sdk`, `aps_automation_sdk`, `requests`
- A local Revit file to upload (SampleModel.rvt)

**Note**: This plugin works specifically with **2D views** in the APS Viewer.

In [1]:
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

# Import from aps_viewer_sdk
from aps_viewer_sdk.helper import get_2lo_token
from aps_viewer_sdk import APSViewer
from aps_viewer_sdk.plugins import Draw2DCircles  # Plugin for adding 2D circle markers

# Import from aps_automation_sdk for uploading and translating models
from aps_automation_sdk import ActivityInputParameter, translate_file_in_oss

load_dotenv()

True

In [2]:
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")

if not CLIENT_ID or not CLIENT_SECRET:
    raise RuntimeError("CLIENT_ID and CLIENT_SECRET must be set in environment")

## Upload and translate the model

Upload the Revit model to APS and translate it for viewing.

In [3]:
from aps_automation_sdk import ActivityInputParameter

client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
if not client_id or not client_secret:
    raise RuntimeError(
        "CLIENT_ID and CLIENT_SECRET must be set in the environment variables."
    )

# Resolve the sample file path
sample_path = Path("SampleModel.rvt").resolve()

print("Starting model processing workflow...")
print(f"Uploading file: {sample_path.name}")

token = get_2lo_token(client_id, client_secret)

bucket_key = uuid.uuid4().hex
object_key = f"input_{uuid.uuid4()}{sample_path.suffix}"

input_param = ActivityInputParameter(
    name="inputFile",
    localName=sample_path.name,
    verb="get",
    description="Input CAD model",
    required=True,
    is_engine_input=False,
    bucketKey=bucket_key,
    objectKey=object_key,
)

input_param.upload_file_to_oss(file_path=str(sample_path), token=token)

# Use the helper from aps_automation_sdk to translate the file
viewer_urn = translate_file_in_oss(
    token=token,
    bucket_key=bucket_key,
    output_object_key=object_key,
    max_wait_time=300,
    poll_interval=15,
    verbose=True,
)
print(f"Viewer URN: {viewer_urn}")

Starting model processing workflow...
Uploading file: SampleModel.rvt
🎯 STARTING MODEL TRANSLATION FOR VIEWING
Bucket: ac674f14aa1440e7ab42df8701cc496f
Object: input_0f333c25-d927-4170-90de-82196427516f.rvt
🔗 Object URN: dXJuOmFkc2sub2JqZWN0czpvcy5vYmplY3Q6YWM2NzRmMTRhYTE0NDBlN2FiNDJkZjg3MDFjYzQ5NmYvaW5wdXRfMGYzMzNjMjUtZDkyNy00MTcwLTkwZGUtODIxOTY0Mjc1MTZmLnJ2dA
🔄 Starting Model Derivative translation job...
📋 Job Details: dXJuOmFkc2sub2JqZWN0czpvcy5vYmplY3Q6YWM2NzRmMTRhYTE0NDBlN2FiNDJkZjg3MDFjYzQ5NmYvaW5wdXRfMGYzMzNjMjUtZDkyNy00MTcwLTkwZGUtODIxOTY0Mjc1MTZmLnJ2dA
⏳ Monitoring translation progress...
  > MD Status: pending (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (56% complete)
  > MD Status: inprogress (65% complete)
  > MD Status: inprogress (74% complete)
  > MD Status: inprogress (94% complete)
  > MD Status: inprogress (99% complete)
  > MD Status: inprogress (99% c

## Configure the Draw2DCircles plugin

The **`Draw2DCircles`** plugin (from `aps_viewer_sdk.plugins`) allows users to add interactive circle markers to 2D views.

### Plugin features:
- **Click to place circles**: Users can click anywhere in a 2D view to add circle markers
- **Customizable appearance**: Set initial radius, color, and opacity
- **2D only**: This plugin is specifically designed for 2D views (floor plans, elevations, sections)

### Plugin parameters:
- `initial_radius`: Default radius for circles (e.g., 0.5)
- `color`: Hex color code for the circles (e.g., "#00ff00" for green)
- `opacity`: Optional transparency level (0.0 to 1.0)


In [4]:
# Create the Draw2DCircles plugin with custom settings
circle_plugin = Draw2DCircles(
    initial_radius=0.5,  # Circle radius in drawing units
    color="#00ff00",  # Green color for circles
)

print("Created Draw2DCircles plugin")
print(f"  - Initial radius: 0.5")
print(f"  - Color: #00ff00 (green)")
print(f"  - Usage: Click on 2D views to place circle markers")

Created Draw2DCircles plugin
  - Initial radius: 0.5
  - Color: #00ff00 (green)
  - Usage: Click on 2D views to place circle markers


## View the model with circle marker plugin

Create the `APSViewer` instance and add the **`Draw2DCircles` plugin** to it.

The plugin is added using `viewer.add_plugin(circle_plugin.spec())`, which injects the interactive circle marker functionality into the viewer. Users can then click on 2D views to place green circle markers at any location.


In [5]:
# Create the viewer with the translated model
viewer = APSViewer(
    urn=f"urn:adsk.objects:os.object:{bucket_key}/{object_key}",
    token=token,
    views_selector=True,  # Enable view selector to switch between 2D/3D views
)

# Add the Draw2DCircles plugin
viewer.add_plugin(circle_plugin.spec())

print("Viewer created with Draw2DCircles plugin")
print("Switch to a 2D view and click to place circle markers!")

# Display the viewer
viewer.show()

Viewer created with Draw2DCircles plugin
Switch to a 2D view and click to place circle markers!


## Result

Here's what the viewer looks like with the 2D circle marker plugin enabled:

![2D Circle Markers Result](../../assets/example3.png)

The `Draw2DCircles` plugin enables interactive annotation on 2D views. Users can click anywhere on floor plans, elevations, or sections to place green circle markers. This is particularly useful for:
- Marking inspection points or issues during reviews
- Annotating specific locations for coordination
- Creating visual indicators for measurements or notes
- Interactive markup and collaboration sessions